In [6]:
import pandas as pd
dataset = pd.read_csv("Social_Network_Ads.csv");
dataset = pd.get_dummies(dataset, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [8]:
dataset = dataset.drop("User ID", axis=1)
dataset = dataset.astype(int)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [10]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [16]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [18]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent = dataset["Purchased"]
independent.shape

(400, 3)

In [24]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(independent, dependent, test_size=1/3, random_state=0)

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], 'penalty': ["l2"]}

grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring="f1_weighted")
grid.fit(xTrain, yTrain)
                    

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [70]:
#print best parameter after tuning
re = grid.cv_results_
#print(re)

grid_predictions = grid.predict(xTest)

In [72]:
#print classification report
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(yTest, grid_predictions)

print(cm)

[[79  6]
 [ 9 40]]


In [74]:
from sklearn.metrics import classification_report
clfReport = classification_report(yTest, grid_predictions)
print(clfReport)

              precision    recall  f1-score   support

           0       0.90      0.93      0.91        85
           1       0.87      0.82      0.84        49

    accuracy                           0.89       134
   macro avg       0.88      0.87      0.88       134
weighted avg       0.89      0.89      0.89       134



In [76]:
from sklearn.metrics import f1_score
f1_macro = f1_score(yTest, grid_predictions, average="macro")
print('The f1_score of the best perameter {}:'.format(grid.best_params_), f1_macro)

The f1_score of the best perameter {'penalty': 'l2', 'solver': 'lbfgs'}: 0.8777000304228779


In [78]:
tabel = pd.DataFrame.from_dict(re)
tabel

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014296,0.003043,0.002445,0.001781,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
1,0.005781,0.003149,0.003200,0.001412,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4
2,0.027085,0.003193,0.001324,0.000243,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
3,0.002105,0.001045,0.001235,0.000676,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
4,0.002742,0.000932,0.001480,0.000761,l2,sag,"{'penalty': 'l2', 'solver': 'sag'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4
5,0.002842,0.001219,0.001288,0.000676,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4


In [56]:
from sklearn.metrics import roc_auc_score
roc_auc_score(yTest, grid.predict_proba(xTest)[:,1])

0.9510204081632653

In [80]:
best_model_row = tabel.loc[tabel['rank_test_score'] == 1].sort_values(by='mean_test_score', ascending=False).iloc[0]
print("Best Parameters:", best_model_row["params"])
print("Best Weighted F1 Score:", best_model_row["mean_test_score"])

Best Parameters: {'penalty': 'l2', 'solver': 'lbfgs'}
Best Weighted F1 Score: 0.8287659267359453
